<a href="https://colab.research.google.com/github/nitinsb/Intern_media/blob/main/Insta_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
folder_path = '/content/drive/MyDrive/Folder 1/Interview December 2023'

In [2]:
if os.path.isdir(folder_path):
  print(f"The path '{folder_path}' exists.")
else:
  print(f"The path '{folder_path}' does not exist.")

The path '/content/drive/MyDrive/Folder 1/Interview December 2023' exists.


In [3]:
# prompt: from this file_path inside info.csv pick the column likes and get me the stats of it

file_path_info = '/content/drive/MyDrive/Folder 1/Interview December 2023/instagram_data.csv'

if os.path.isfile(file_path_info):
  try:
    df = pd.read_csv(file_path_info)
    likes_stats = df['likes'].describe()
    print(likes_stats)
  except Exception as e:
    print(f"An error occurred: {e}")
else:
  print(f"The file '{file_path_info}' does not exist.")


count    3.785000e+03
mean     1.832536e+05
std      1.936969e+05
min      1.431000e+03
25%      5.208700e+04
50%      1.236640e+05
75%      2.431440e+05
max      2.161369e+06
Name: likes, dtype: float64


In [ ]:
# Calculate the lower and upper bounds for outliers based on IQR
Q1 = df['likes'].quantile(0.05)
Q3 = df['likes'].quantile(0.95)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
df_no_outliers = df[(df['likes'] >= lower_bound) & (df['likes'] <= upper_bound)]

# Print the shape of the original and outlier-removed dataframes
print(f"Original DataFrame shape: {df.shape}")
print(f"DataFrame shape after removing outliers: {df_no_outliers.shape}")


In [ ]:
X = df_no_outliers[['likes','no_of_comments', 't', 'follower_count_at_t','image_path']]
X.shape


In [ ]:
print(X['likes'].describe())


In [ ]:


# Define the thresholds for the 3-way classification
threshold_low = X['likes'].quantile(0.33)
threshold_high = X['likes'].quantile(0.66)

# Create the new column 'y' based on the thresholds
def classify_likes(likes):
  if likes <= threshold_low:
    return 0
  elif likes <= threshold_high:
    return 1
  else:
    return 2

X['y'] = X['likes'].apply(classify_likes)

print(X['y'].value_counts())


In [ ]:


print(X.head(50))
# the class has good distrubution

In [ ]:
X_y = X.drop('likes', axis=1)
X_y.head(50)

In [ ]:
%cd /content/drive/MyDrive/Folder 1/Interview December 2023/

!pip install tensorflow
!pip install keras
!pip install keras_applications

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:



# Load a pre-trained CNN model (ResNet50 in this example)
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def get_image_embedding(image_path):
  try:
    img = image.load_img(image_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    return features.flatten()
  except Exception as e:
    print(f"Error processing image {image_path}: {e}")
    return None

# Example usage:
image_embeddings = []
for image_path in X_y['image_path']:
  print(image_path)
  embedding = get_image_embedding(image_path)
  if embedding is not None:
    image_embeddings.append(embedding)
  else:
    image_embeddings.append([0] * 2048) # Or handle missing embeddings as needed

# Add the embeddings to the dataframe
X_y['image_embedding'] = image_embeddings

print(X_y.head())


In [18]:


X_y_emb = X_y.drop('image_path', axis=1)
X_y_emb.head(50)


,no_of_comments,t,follower_count_at_t,y,image_embedding
0,0,1594174009,40934474,1,"[1.06547, 0.5881613, 0.44351718, 0.44114748, 0..."
1,0,1593571666,40934474,1,"[0.58651054, 0.36541268, 1.1357396, 1.1676589,..."
2,0,1593136341,40934474,1,"[0.4688445, 1.5273312, 0.29110304, 0.0693633, ..."
3,0,1592981047,40934474,1,"[0.82497185, 0.38535434, 0.2783496, 0.4984197,..."
4,0,1592703461,40934474,1,"[0.004814681, 0.08772654, 0.10228238, 0.030042..."
5,0,1592276125,40934474,1,"[0.27827355, 0.58995146, 0.07432286, 0.1422063..."
6,0,1591582341,40934474,1,"[0.6458573, 2.0152388, 0.32338053, 1.2982101, ..."
7,0,1589702043,40934474,2,"[0.014141029, 0.5951965, 0.053449094, 1.108684..."
8,0,1588470412,40934474,2,"[1.0093479, 0.15728326, 0.07973807, 0.0, 0.313..."
9,0,1587696339,40934474,2,"[0.42805022, 1.6021767, 0.028481599, 0.695731,..."


In [19]:


from sklearn.model_selection import train_test_split

X = X_y_emb.drop('y', axis=1)
y = X_y_emb['y']

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_test shape: {Y_test.shape}")


X_train shape: (3023, 4)
Y_train shape: (3023,)
X_test shape: (756, 4)
Y_test shape: (756,)


In [21]:


from sklearn.preprocessing import StandardScaler


numerical_features = ['no_of_comments', 't', 'follower_count_at_t']


scaler = StandardScaler()

X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])



print(X_train.head())
print(X_test.head())


      no_of_comments         t  follower_count_at_t  \
3552       -0.090845 -1.409514            -0.978271   
543        -0.108466  0.642249             1.003118   
2365       -0.103556  0.445742            -0.522049   
3319        0.042372  0.387155            -0.737797   
1615       -0.051334 -0.802690             0.038099   

                                        image_embedding  
3552  [0.5091548, 0.3770356, 0.0, 0.36231288, 1.1243...  
543   [0.20883377, 0.65287936, 0.027432326, 3.893399...  
2365  [0.0076132207, 0.0, 0.037819117, 0.0, 0.0, 0.0...  
3319  [0.6514008, 1.4751356, 0.0037992073, 2.7833176...  
1615  [0.0674658, 0.44933435, 0.12690702, 1.6582507,...  
      no_of_comments         t  follower_count_at_t  \
1292        0.381860 -2.125342            -0.017016   
2620       -0.104611 -0.022089            -0.696068   
2361       -0.092267  0.669443            -0.522049   
298        -0.109200 -0.226284             1.517566   
1126       -0.110761  0.703900             0.2

In [25]:


import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, Flatten
from tensorflow.keras.optimizers import Adam


numerical_inputs = Input(shape=(len(numerical_features),))
image_embedding_input = Input(shape=(2048,))


numerical_dense = Dense(128, activation='relu')(numerical_inputs)


merged_layer = concatenate([numerical_dense, image_embedding_input])


dense1 = Dense(64, activation='relu')(merged_layer)
output_layer = Dense(3, activation='softmax')(dense1)

model = Model(inputs=[numerical_inputs, image_embedding_input], outputs=output_layer)
model.summary()

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

X_train_numerical = X_train[numerical_features]
X_train_image_embeddings = np.array(X_train['image_embedding'].tolist())
X_test_numerical = X_test[numerical_features]
X_test_image_embeddings = np.array(X_test['image_embedding'].tolist())

# Train the model
model.fit([X_train_numerical, X_train_image_embeddings], Y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate([X_test_numerical, X_test_image_embeddings], Y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 128)            │            512 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_8             │ (None, 2048)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 2176)           │              0 │ dense_6[0][0],         │
│ (Concatenate)             │                        │                │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 64)             │        139,328 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 3)              │            195 │ dense_7[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 140,035 (547.01 KB)

 Trainable params: 140,035 (547.01 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.3842 - loss: 1.3461 - val_accuracy: 0.4496 - val_loss: 1.0206
Epoch 2/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4861 - loss: 1.0151 - val_accuracy: 0.4711 - val_loss: 1.0136
Epoch 3/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5468 - loss: 0.9303 - val_accuracy: 0.4826 - val_loss: 0.9848
Epoch 4/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5966 - loss: 0.8606 - val_accuracy: 0.5207 - val_loss: 0.9888
Epoch 5/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6579 - loss: 0.7768 - val_accuracy: 0.5273 - val_loss: 0.9719
Epoch 6/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7359 - loss: 0.6661 - val_accuracy: 0.5190 - val_loss: 1.0308
Epoch 7/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7686 - loss: 0.5930 - val_accuracy: 0.5537 - val_loss: 0.9933
Epoch 8/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8170 - loss: 0.4818 - val_accuracy: 0.5273 - val_loss

In [27]:


numerical_inputs = Input(shape=(len(numerical_features),))
image_embedding_input = Input(shape=(2048,))


numerical_dense = Dense(128, activation='relu')(numerical_inputs)


merged_layer = concatenate([numerical_dense, image_embedding_input])


dense1 = Dense(64, activation='relu')(merged_layer)
dropout_layer = tf.keras.layers.Dropout(0.5)(dense1) # Add a dropout layer to prevent overfitting
output_layer = Dense(3, activation='softmax')(dropout_layer)

model = Model(inputs=[numerical_inputs, image_embedding_input], outputs=output_layer)
model.summary()

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

X_train_numerical = X_train[numerical_features]
X_train_image_embeddings = np.array(X_train['image_embedding'].tolist())
X_test_numerical = X_test[numerical_features]
X_test_image_embeddings = np.array(X_test['image_embedding'].tolist())

# Train the model
model.fit([X_train_numerical, X_train_image_embeddings], Y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate([X_test_numerical, X_test_image_embeddings], Y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11            │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 128)            │            512 │ input_layer_11[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_12            │ (None, 2048)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_4             │ (None, 2176)           │              0 │ dense_12[0][0],        │
│ (Concatenate)             │                        │                │ input_layer_12[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 64)             │        139,328 │ concatenate_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 64)             │              0 │ dense_13[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_14 (Dense)          │ (None, 3)              │            195 │ dropout_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 140,035 (547.01 KB)

 Trainable params: 140,035 (547.01 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.3650 - loss: 1.3433 - val_accuracy: 0.3901 - val_loss: 1.0680
Epoch 2/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4082 - loss: 1.0591 - val_accuracy: 0.4579 - val_loss: 1.0353
Epoch 3/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4659 - loss: 1.0295 - val_accuracy: 0.4711 - val_loss: 1.0183
Epoch 4/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4823 - loss: 0.9907 - val_accuracy: 0.4876 - val_loss: 1.0053
Epoch 5/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4804 - loss: 0.9854 - val_accuracy: 0.4760 - val_loss: 0.9922
Epoch 6/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4853 - loss: 0.9656 - val_accuracy: 0.4992 - val_loss: 0.9904
Epoch 7/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5151 - loss: 0.9349 - val_accuracy: 0.4826 - val_loss: 0.9800
Epoch 8/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5262 - loss: 0.9134 - val_accuracy: 0.5025 - 

In [ ]:


!pip install transformers

from transformers import CLIPProcessor, CLIPModel
import torch

# Load the CLIP model and processor
model_clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor_clip = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def get_clip_embedding(image_path):
  try:
    image = image.load_img(image_path)
    inputs = processor_clip(images=image, return_tensors="pt")
    with torch.no_grad():
      outputs = model_clip.get_image_features(**inputs)
    return outputs.numpy().flatten()
  except Exception as e:
    print(f"Error processing image {image_path}: {e}")
    return None

# Example usage:
clip_embeddings = []
for image_path in X_y['image_path']:
  print(image_path)
  embedding = get_clip_embedding(image_path)
  if embedding is not None:
    clip_embeddings.append(embedding)
  else:
    clip_embeddings.append([0] * 512) # Or handle missing embeddings as needed

# Add the embeddings to the dataframe
X_y['clip_embedding'] = clip_embeddings

print(X_y.head())



X_y_emb_clip = X_y.drop('image_embedding', axis=1)
X_y_emb_clip = X_y_emb_clip.drop('image_path', axis=1)
X_y_emb_clip.head(50)


X = X_y_emb_clip.drop('y', axis=1)
y = X_y_emb_clip['y']

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_test shape: {Y_test.shape}")





numerical_features = ['no_of_comments', 't', 'follower_count_at_t']


scaler = StandardScaler()

X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])



print(X_train.head())
print(X_test.head())





numerical_inputs = Input(shape=(len(numerical_features),))
clip_embedding_input = Input(shape=(512,))


numerical_dense = Dense(128, activation='relu')(numerical_inputs)


merged_layer = concatenate([numerical_dense, clip_embedding_input])


dense1 = Dense(64, activation='relu')(merged_layer)
dropout_layer = tf.keras.layers.Dropout(0.5)(dense1) # Add a dropout layer to prevent overfitting
output_layer = Dense(3, activation='softmax')(dropout_layer)

model = Model(inputs=[numerical_inputs, clip_embedding_input], outputs=output_layer)
model.summary()

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

X_train_numerical = X_train[numerical_features]
X_train_clip_embeddings = np.array(X_train['clip_embedding'].tolist())
X_test_numerical = X_test[numerical_features]
X_test_clip_embeddings = np.array(X_test['clip_embedding'].tolist())

# Train the model
model.fit([X_train_numerical, X_train_clip_embeddings], Y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate([X_test_numerical, X_test_clip_embeddings], Y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
